# **Ready & Data load**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install torch_geometric

In [ ]:
import pandas as pd
import numpy as np

from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric import utils
import torch
import os
import random
from tqdm import tqdm
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d
from torch_geometric.nn import TransformerConv, TopKPooling, global_mean_pool
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, average_precision_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import unicodedata

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class EEGDataset(Dataset):
    def __init__(self, root, edge_dict, node_list = None, label_list = [0,1,2,3], threshold = -100, transform = None, pre_transform = None):
        """
        root = 데이터셋이 저장되어야 할 루트 디렉토리.
        상위폴더 속 raw = tabular data 저장되어 있음.
        하위폴더 processed = EEGDataset 실행시 자동 생성. 이미 생성되어 있을 경우 process 함수 자동 생략 및 빠르게 실행됨.
        """
        # 데이터셋의 루트 디렉토리를 지정
        self.root = root
        # 사용할 엣지 리스트 입력
        self.edge_dict = edge_dict
        # 사용할 노드 리스트 입력
        self.node_list = node_list
        # 사용할 라벨 리스트 입력
        self.label_list = label_list
        # threshold 입력
        self.threshold = threshold

        # 라벨과 파일 경로 매핑을 저장할 리스트
        self.data_info = self._load_data_info()

        # 부모 클래스 초기화
        super(EEGDataset, self).__init__(root, transform, pre_transform)

    def _load_data_info(self):
        """
        raw_dir에 있는 라벨별 폴더를 탐색하여 파일 경로와 라벨 정보를 매핑.
        """
        data_info = []

        # raw_dir에 대한 경로 설정
        raw_dir = os.path.join(os.path.dirname(self.root), "raw_split")

        # 라벨별 폴더에서 파일 찾기
        for split_folder in os.listdir(raw_dir):
            split_path = os.path.join(raw_dir, split_folder)
            for label_folder in os.listdir(split_path):
                if int(label_folder[0]) in self.label_list:
                    label_path = os.path.join(split_path, label_folder)
                    if os.path.isdir(label_path):
                        # 각 라벨 폴더의 파일을 살펴봄
                        for filename in os.listdir(label_path):
                            file_path = os.path.join(label_path, filename)
                            if os.path.isfile(file_path):
                                # 파일 경로와 라벨 매핑
                                data_info.append((file_path, label_folder, split_folder))

        return data_info

    @property
    def raw_file_names(self):
        # raw_dir 안에 있는 모든 txt 파일 경로를 반환
        return [info[0] for info in self.data_info]

    @property
    def processed_file_names(self):
        # data_info에 있는 각 파일 경로에서 파일 이름을 추출하고, 확장자를 제거합니다.
        processed_file_names = []
        for file_path, _, _ in self.data_info:
            # 파일 경로에서 파일 이름 추출
            filename = os.path.basename(file_path)
            # 확장자 제거
            base_filename, _ = os.path.splitext(filename)
            # 처리된 파일 이름 생성
            processed_file_names.append(f"{base_filename}.pt")

        return processed_file_names

    def process(self):
        # 데이터셋 처리
        for idx, (file_path, label, split) in enumerate(tqdm(self.data_info)):
            # label 처리
            label = self._get_labels(label)

            # 파일 읽기
            df = self._read_data(file_path, label)

            # 그래프 데이터 변환 정보
            node_features = self._get_node_features(df)
            edge_index = self._get_edge_indices(df)
            edge_features = self._get_edge_features(df)

            # 그래프 데이터 생성
            data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features, y=label)

            # test data 여부
            data.test_mask = self._get_test_mask(split)

            # Data 객체 저장
            base_filename = os.path.splitext(os.path.basename(file_path))[0]
            torch.save(data, os.path.join(self.processed_dir, f"{base_filename}.pt"))

    def _read_data(self, file_path, label):
        # 파일을 탭으로 구분된 값으로 읽기
        df = pd.read_csv(file_path, sep="\t")
        # 데이터프레임의 일부 행과 열 추출
        df = df.iloc[1:3829, 0:41]
        # 열 이름 설정
        df.columns = ['brain_region'] + [f"freq{i}" for i in range(1, 41)]
        # 엣지 리스트에 해당하는 정보만 사용
        df = df.loc[df['brain_region'].isin(self.edge_dict[int(label)])]
        # wave 변환
        df['delta'] = df[[f"freq{i}" for i in range(1, 4)]].mean(axis=1)
        df['theta'] = df[[f"freq{i}" for i in range(4, 9)]].mean(axis=1)
        df['alpha'] = df[[f"freq{i}" for i in range(9, 14)]].mean(axis=1)
        df['beta'] = df[[f"freq{i}" for i in range(14, 31)]].mean(axis=1)
        df['gamma'] = df[[f"freq{i}" for i in range(14, 41)]].mean(axis=1)
        df = df[['brain_region', 'delta', 'theta', 'alpha', 'beta', 'gamma']]

        # threshold 처리
        waves = ['delta', 'theta', 'alpha', 'beta', 'gamma']
        df[waves] = df[waves].map(lambda x: 0 if x < self.threshold else x)
        df = df[~(df[waves].sum(axis=1) == 0)]

        return df

    def _get_node_features(self, df):
        # 노드 리스트가 지정된 경우, 노드 리스트에 해당하는 노드 사용
        if self.node_list is not None:
            unique_brain_regions = self.node_list
        # 노드 리스트가 지정되지 않은 경우, 엣지에 사용되는 노드 사용
        else:
            unique_brain_regions = pd.unique(df['brain_region'].str.split(' ', expand=True).stack())
        # 노드의 숫자 변환 정보 딕셔너리에 저장
        self.brain_region_to_index = {region: idx for idx, region in enumerate(unique_brain_regions)}
        # node 특성 1로 채우기
        node_features = np.ones(shape=(len(unique_brain_regions),1))

        return torch.tensor(node_features, dtype=torch.float)

    def _get_edge_indices(self, df):
        # edge index 추출
        edge_index = df['brain_region'].str.split(' ', expand=True).values.transpose()
        # 엣지를 이루는 노드 정보를 숫자로 변환
        numerical_edge_index = np.vectorize(self.brain_region_to_index.get)(edge_index)

        return torch.tensor(numerical_edge_index, dtype=torch.long)

    def _get_edge_features(self, df):
        edge_features = df.drop(columns=['brain_region']).values

        return torch.tensor(edge_features, dtype=torch.float)

    def _get_labels(self, label):
        label_num = np.asarray([int(label[0])])

        return torch.tensor(label_num, dtype=torch.long)

    def _get_test_mask(self, split):
        test_mask = 1 if split == 'test' else 0
        test_mask = np.asarray([test_mask])

        return torch.tensor(test_mask, dtype=torch.long)

    def len(self):
        # 데이터셋의 길이 반환

        return len(self.data_info)

    def get(self, idx):
        # 인덱스에 해당하는 데이터를 로드하여 반환
        file_path, _, _ = self.data_info[idx]
        base_filename = os.path.splitext(os.path.basename(file_path))[0]

        return torch.load(os.path.join(self.processed_dir, f"{base_filename}.pt"))

# **Modeling & Utils**

In [ ]:
# 시드 고정 함수
def set_seed(seed):
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True # 딥러닝에 특화된 CuDNN의 난수시드도 고정
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed) # 파이썬 자체 모듈 random 모듈의 시드 고정

In [ ]:
# modeling
class GraphClassifier(torch.nn.Module):
      def __init__(self, in_channels, hidden_channels, out_channels, edge_dim):
          super(GraphClassifier, self).__init__()
          self.conv1 = TransformerConv(in_channels, hidden_channels, heads=n_heads, edge_dim=edge_dim)
          self.conv2 = TransformerConv(hidden_channels, hidden_channels, heads=n_heads, edge_dim=edge_dim)
          self.conv3 = TransformerConv(hidden_channels, hidden_channels, heads=n_heads, edge_dim=edge_dim)
          self.transf = torch.nn.Linear(hidden_channels*n_heads, hidden_channels)
          self.bn = BatchNorm1d(hidden_channels)
          self.pool = TopKPooling(hidden_channels, ratio=top_k_ratio)
          self.lin = torch.nn.Linear(hidden_channels, out_channels)

      def forward(self, x, edge_index, edge_attr, batch):
          x = self.conv1(x, edge_index, edge_attr)
          x = F.relu(self.transf(x))
          x = self.bn(x)

          x = self.conv2(x, edge_index, edge_attr)
          x = F.relu(self.transf(x))
          x = self.bn(x)

          x = self.conv3(x, edge_index, edge_attr)
          x = F.relu(self.transf(x))
          x = self.bn(x)

          x, edge_index, edge_attr, batch, _, _ = self.pool(x, edge_index, edge_attr, batch)
          x = global_mean_pool(x, batch)
          x = F.dropout(x, p=dropout_rate, training=self.training)
          x = self.lin(x)
          x = F.log_softmax(x, dim=1)
          return x

In [ ]:
# 그래프 데이터용 층화 분리 함수
def stratified_kfold_split(dataset, n_splits):
    indices = range(len(dataset))
    labels = [dataset[i].y.item() for i in indices]

    skf = StratifiedKFold(n_splits = n_splits)
    return skf.split(indices, labels)

In [ ]:
def split_with_mask(dataset):
    train_idx = [i for i in range(len(dataset)) if dataset[i]['test_mask'] == 0]
    test_idx = [i for i in range(len(dataset)) if dataset[i]['test_mask'] == 1]

    train_dataset = dataset[train_idx]
    test_dataset = dataset[test_idx]

    return train_dataset, test_dataset

In [ ]:
def train():
    best_val_auc = 0
    train_auc_trace = list()
    val_auc_trace = list()

    for epoch in range(num_epochs):
        set_seed(42 + epoch)
        print('\n', '>> Epoch: ', str(epoch))

        epoch_train_aucs = []
        epoch_val_aucs = []

        # 5-CV
        for fold, (train_idx, val_idx) in enumerate(stratified_kfold_split(train_dataset, 5)):
            train_loader = DataLoader(train_dataset, batch_size, sampler=torch.utils.data.SubsetRandomSampler(train_idx))
            val_loader = DataLoader(train_dataset, batch_size, sampler=torch.utils.data.SubsetRandomSampler(val_idx))

            # Train loop
            model.to(device)
            model.train()

            train_preds = []
            train_labels = []
            train_losses = []
            train_preds_prob = []

            for data in train_loader:
                data = data.to(device)
                optimizer.zero_grad()
                out = model(data.x, data.edge_index, data.edge_attr, data.batch)
                prob = torch.softmax(out, dim=1)[:, 1]
                loss = F.nll_loss(out, data.y.long())
                loss.backward()
                optimizer.step()

                train_losses.append(loss.item())
                train_preds_prob.extend(prob.detach().cpu().numpy())
                train_labels.extend(data.y.cpu().numpy())

            train_auc = roc_auc_score(train_labels, train_preds_prob)
            epoch_train_aucs.append(train_auc)

            # Validation loop
            model.eval()
            val_preds = []
            val_labels = []
            val_losses = []
            val_preds_prob = []

            with torch.no_grad():
                for data in val_loader:
                    data = data.to(device)
                    out = model(data.x, data.edge_index, data.edge_attr, data.batch)
                    prob = torch.softmax(out, dim=1)[:, 1]
                    loss = F.nll_loss(out, data.y.long())

                    val_losses.append(loss.item())
                    val_preds_prob.extend(prob.detach().cpu().numpy())
                    val_labels.extend(data.y.cpu().numpy())

            val_auc = roc_auc_score(val_labels, val_preds_prob)
            epoch_val_aucs.append(val_auc)
            print(f'Fold {fold} - Train AUC: {train_auc:.4f}, Validation AUC: {val_auc:.4f}')

        # Calculate average AUC for this epoch across folds
        avg_train_auc = np.mean(epoch_train_aucs)
        avg_val_auc = np.mean(epoch_val_aucs)
        train_auc_trace.append(avg_train_auc)
        val_auc_trace.append(avg_val_auc)

        print(f'Epoch {epoch} - Average Train AUC: {avg_train_auc:.4f}, Average Validation AUC: {avg_val_auc:.4f}')

        # Save the best model
        if avg_val_auc > best_val_auc:
            best_val_auc = avg_val_auc
            save_path = os.path.join(save_dir, label_name + '_weight_test_auc_' + f'{best_val_auc:.4f}' + '.pt')
            normalized_save_path = unicodedata.normalize('NFC', save_path)
            torch.save(model.state_dict(), normalized_save_path)
            print(f'> Best model saved with Validation AUC: {best_val_auc:.4f} <')

    # Plotting
    epochs = range(1, num_epochs + 1)
    plt.figure(figsize=(12, 6))
    plt.plot(epochs, train_auc_trace, 'orange', label='Train AUC')
    plt.plot(epochs, val_auc_trace, 'lightblue', label='Validation AUC')
    plt.title('Train and Validation AUC Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# test 함수
def test():
    set_seed(42)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

    model_test.to(device)
    model_test.eval()
    test_preds = list()
    test_labels = list()
    test_losses = list()
    test_preds_prob = list()

    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model_test(data.x, data.edge_index, data.edge_attr, data.batch)
            pred = out.argmax(dim=1)
            prob = torch.softmax(out, dim=1)[:, 1]
            loss = F.nll_loss(out, data.y.long())

            test_losses.append(loss.item())
            test_preds.extend(pred.cpu().numpy())
            test_labels.extend(data.y.cpu().numpy())
            test_preds_prob.extend(prob.detach().cpu().numpy())

    return test_losses, test_preds, test_labels, test_preds_prob

In [ ]:
label_numlist_dict = {'건강조울':[0, 1], '건강우울':[0, 2], '건강조현':[0, 3],
                      '조울우울':[1, 2], '조울조현':[1, 3], '우울조현':[2, 3], '건강질환':[0, 1, 2, 3, 4],
                      '건강조울우울':[0, 1, 2], '건강조울조현':[0, 1, 3], '건강우울조현':[0, 2, 3], '조울우울조현':[1, 2, 3],
                      '건강조울우울조현':[0, 1, 2, 3]}

In [ ]:
label_title_dict = {'건강조울':'Healthy controls vs. Bipolar disorder',
                    '건강우울':'Healthy controls vs. Depressive disorder',
                    '건강조현':'Healthy controls vs. Schizophrenia',
                    '조울우울':'Bipolar disorder vs. Depressive disorder',
                    '조울조현':'Bipolar disorder vs. Schizophrenia',
                    '우울조현':'Depressive disorder vs. Schizophrenia',
                    '건강질환':'Healthy controls vs. Major psychiatric disorder',
                    '건강조울우울':'Healthy controls vs. Bipolar disorder vs. Depressive disorder',
                    '건강조울조현':'Healthy controls vs. Bipolar disorder vs. Schizophrenia',
                    '건강우울조현':'Healthy controls vs. Depressive disorder vs. Schizophrenia',
                    '조울우울조현':'Bipolar disorder vs. Depressive disorder vs. Schizophrenia',
                    '건강조울우울조현':'Healthy controls vs. Bipolar disorder vs. Depressive disorder vs. Schizophrenia'}

In [ ]:
# AUC 계산 및 ROC curve 시각화 함수
def draw_roc_curve():
    fpr, tpr, thresholds = metrics.roc_curve(test_labels, test_preds_prob)
    roc_auc = metrics.auc(fpr, tpr)

    plt.figure(facecolor="w",figsize=(8,8))

    plt.plot([0,1], [0,1], linestyle='--', markersize=0.01, color='black')
    plt.plot(fpr, tpr, marker='.', color='green', linewidth=3, markersize=0.05, label="AUC = %.3f" % roc_auc)

    plt.xlabel('False Positive Rate', fontsize=18)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.title(label_title_dict[label_name], fontdict = {'fontsize' : 20})
    plt.legend(loc=4, fontsize=18)

    plt.show()

# **Training & Evaluation**

In [ ]:
trial_num = '1118_1'

## **1. Bonferroni (0.05)**


In [ ]:
correction_type = 'bonf_0.05'

In [ ]:
save_dir = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/model/{trial_num}/{correction_type}"

In [ ]:
# 하이퍼파라미터
batch_size = 8
hidden_channels = 16
learning_rate = 0.01
num_epochs = 100
n_heads = 4
top_k_ratio = 0.5
dropout_rate = 0.5

### **1-1. Healthy controls vs. Bipolar disorder**

In [ ]:
# 라벨 지정
label_name = '건강조울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **1-2. Healthy controls vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '건강우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **1-3. Healthy controls vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '건강조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **1-4. Bipolar disorder vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '조울우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **1-5. Bipolar disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '조울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **1-6. Depressive disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '우울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **1-7. Healthy controls vs. Major psychiatric disorder**

In [ ]:
# 라벨 지정
label_name = '건강질환'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

## **2. Bonferroni (0.01)**


In [ ]:
correction_type = 'bonf_0.01'

In [ ]:
save_dir = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/model/{trial_num}/{correction_type}"

In [ ]:
# 하이퍼파라미터
batch_size = 8
hidden_channels = 16
learning_rate = 0.01
num_epochs = 100
n_heads = 4
top_k_ratio = 0.5
dropout_rate = 0.5

### **2-1. Healthy controls vs. Bipolar disorder**

In [ ]:
# 라벨 지정
label_name = '건강조울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **2-2. Healthy controls vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '건강우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **2-3. Healthy controls vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '건강조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **2-4. Bipolar disorder vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '조울우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **2-5. Bipolar disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '조울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **2-6. Depressive disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '우울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **2-7. Healthy controls vs. Major psychiatric disorder**

In [ ]:
# 라벨 지정
label_name = '건강질환'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

## **3. Benjamini-Hochberg (0.05)**


In [ ]:
correction_type = 'bh_0.05'

In [ ]:
save_dir = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/model/{trial_num}/{correction_type}"

In [ ]:
# 하이퍼파라미터
batch_size = 8
hidden_channels = 16
learning_rate = 0.01
num_epochs = 100
n_heads = 4
top_k_ratio = 0.5
dropout_rate = 0.5

### **3-1. Healthy controls vs. Bipolar disorder**

In [ ]:
# 라벨 지정
label_name = '건강조울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **3-2. Healthy controls vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '건강우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **3-3. Healthy controls vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '건강조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **3-4. Bipolar disorder vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '조울우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **3-5. Bipolar disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '조울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **3-6. Depressive disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '우울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **3-7. Healthy controls vs. Major psychiatric disorder**

In [ ]:
# 라벨 지정
label_name = '건강질환'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

## **4. Benjamini-Hochberg (0.01)**


In [ ]:
correction_type = 'bh_0.01'

In [ ]:
save_dir = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/model/{trial_num}/{correction_type}"

In [ ]:
# 하이퍼파라미터
batch_size = 8
hidden_channels = 16
learning_rate = 0.01
num_epochs = 100
n_heads = 4
top_k_ratio = 0.5
dropout_rate = 0.5

### **4-1. Healthy controls vs. Bipolar disorder**

In [ ]:
# 라벨 지정
label_name = '건강조울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **4-2. Healthy controls vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '건강우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **4-3. Healthy controls vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '건강조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **4-4. Bipolar disorder vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '조울우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **4-5. Bipolar disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '조울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **4-6. Depressive disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '우울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **4-7. Healthy controls vs. Major psychiatric disorder**

In [ ]:
# 라벨 지정
label_name = '건강질환'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

## **5. Benjamini-Yekutieli (0.05)**


In [ ]:
correction_type = 'by_0.05'

In [ ]:
save_dir = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/model/{trial_num}/{correction_type}"

In [ ]:
# 하이퍼파라미터
batch_size = 8
hidden_channels = 16
learning_rate = 0.01
num_epochs = 100
n_heads = 4
top_k_ratio = 0.5
dropout_rate = 0.5

### **5-1. Healthy controls vs. Bipolar disorder**

In [ ]:
# 라벨 지정
label_name = '건강조울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **5-2. Healthy controls vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '건강우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **5-3. Healthy controls vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '건강조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **5-4. Bipolar disorder vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '조울우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **5-5. Bipolar disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '조울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **5-6. Depressive disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '우울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **5-7. Healthy controls vs. Major psychiatric disorder**

In [ ]:
# 라벨 지정
label_name = '건강질환'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

## **6. Benjamini-Yekutieli (0.01)**


In [ ]:
correction_type = 'by_0.01'

In [ ]:
save_dir = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/model/{trial_num}/{correction_type}"

In [ ]:
# 하이퍼파라미터
batch_size = 8
hidden_channels = 16
learning_rate = 0.01
num_epochs = 100
n_heads = 4
top_k_ratio = 0.5
dropout_rate = 0.5

### **6-1. Healthy controls vs. Bipolar disorder**

In [ ]:
# 라벨 지정
label_name = '건강조울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **6-2. Healthy controls vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '건강우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **6-3. Healthy controls vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '건강조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **6-4. Bipolar disorder vs. Depressive disorder**

In [ ]:
# 라벨 지정
label_name = '조울우울'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **6-5. Bipolar disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '조울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **6-6. Depressive disorder vs. Schizophrenia**

In [ ]:
# 라벨 지정
label_name = '우울조현'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()

### **6-7. Healthy controls vs. Major psychiatric disorder**

In [ ]:
# 라벨 지정
label_name = '건강질환'

In [ ]:
# 데이터 불러오기
edge_df = pd.read_csv(f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/edge_selection/{label_name}.csv")
edge_list = edge_df.loc[edge_df[correction_type],'brain_region'].tolist()

dataset = EEGDataset(root = f"/content/drive/Othercomputers/내 노트북/2024-1/EEG_GNN/data/{label_name}_{correction_type}",
                     edge_dict = {num: edge_list for num in label_numlist_dict[label_name]},
                     label_list = label_numlist_dict[label_name])

In [ ]:
# train, test 분리
train_dataset, test_dataset = split_with_mask(dataset)

In [ ]:
# 모델 초기화
model = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
train()

In [ ]:
# 저장된 모델 불러오기
weight_path = next(os.path.join(save_dir, unicodedata.normalize('NFC', filename)) for filename in sorted(os.listdir(save_dir), reverse=True) if unicodedata.normalize('NFC', filename).startswith(label_name))
print(weight_path)

In [ ]:
# 모델 초기화
model_test = GraphClassifier(
    in_channels = dataset.num_node_features,
    hidden_channels = hidden_channels,
    out_channels = dataset.num_classes,
    edge_dim = dataset.num_edge_features
)

model_test.load_state_dict(torch.load(weight_path))

In [ ]:
# 실행
test_losses, test_preds, test_labels, test_preds_prob = test()

In [ ]:
# AUC 확인 및 ROC curve 시각화
draw_roc_curve()